# Common REST API Demo

[Common REST API Docs](https://enclave-markets.notion.site/Common-REST-API-9d546fa6282b4bad87ef43d189b9071b)

## getting started

Note that for many of the endpoints such as deposit and withdrawal to work, you cannot use the sandbox environment.

### auth

Before using, please read the [overview docs](https://enclave-markets.notion.site/REST-API-Overview-57966a627e5445bba573fd66475a553d)
as well as the [authentication docs](https://enclave-markets.notion.site/Rest-API-Authentication-3956dcfecbdc48269031cf052926c90d) 
to get an API key with View + Transfer to use the full demo, or just make a key with View to ensure no funds are moved.

### prepare account

After following the instruction to get an API key and secret, for all the endpoints to work as shown, do the following:

1.  provision a deposit address for AVAX
2.  deposit (not from a smart wallet!) some AVAX to the address
    1.  save the transaction ID from the chain.
3.  add a withdrawal address to your address book

now you're all set :)


## setup

Install dependencies. 

### janky fix import

And add the enclave module's path to the system path so we can import the local module in jupyter. 
Alternatively in VSCode you can follow [this tutorial](https://stackoverflow.com/a/73954768),
or from the command line generally [this tutorial](https://stackoverflow.com/a/65182353).

If you follow the above, feel free to remove the next cell.

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### dependencies

Continue installing and importing dependencies normally.

In [ ]:
%pip install requests

import enclave

## environment variables

Set the following variables for auth and to set the base url.

In [ ]:
import enclave.models
from IPython.core.error import UsageError

# change to the env of your choice that matches the API keys: PROD, DEV, SANDBOX etc.
BASE_URL = enclave.models.SANDBOX


### three ways to get API key and secret, tried in order.

# 1. as hardcoded strings
API_KEY: str = ""
API_SECRET: str = ""

# 2. as environment variables
if not(all([API_KEY, API_SECRET])):
    try:
        API_KEY = %env ENCLAVE_API_KEY
        API_SECRET = %env ENCLAVE_API_SECRET
    except UsageError:
        pass

# 3. from a .env file with `key` and `secret` set.
if not(all([API_KEY, API_SECRET])):
    %pip install python-dotenv
    import dotenv
    envs = dotenv.dotenv_values()
    if envs and "key" in envs and "secret" in envs:
        API_KEY, API_SECRET = str(envs["key"]), str(envs["secret"])


if not(all([API_KEY, API_SECRET])):
    raise ValueError("Please provide API_KEY and API_SECRET")

## create client

Create a client with the variables and get authed hello.

If you get anything other than a 200 status code and the hello message with `"success": True` 
and your Enclave Account ID, then there is an issue -- <br>
either with auth or connection permissions to the URL.

In [ ]:
from enclave.client import Client

client = Client(api_key=API_KEY, api_secret=API_SECRET, base_url=BASE_URL)

hello_res = client.authed_hello()
if 200 <= hello_res.status_code < 300:
    print(hello_json := hello_res.json())
    assert hello_json["success"] == True
else:
    print(f"Could not connect:\n{hello_res.text=}")